In [1]:
import numpy as np
from matplotlib import pyplot as plt
import math
import scipy
import lasio
import os
import pandas as pd

path = r'./welldata/RIGIS/'

In [7]:
lasDataFrame = []
for file in os.listdir(path):
    if file.endswith('.las'):
        lasDataFrame.append(lasio.read(path+file).df())
lasDataFrame[0].style

,NAS_T,LIT_T,KP_T,KG_T,KPR_T:1,KPR_T:2
DEPT,,,,,,
824.000000,nan,nan,nan,nan,nan,4.606076
825.000000,nan,0.000000,nan,nan,nan,nan
826.000000,0.000000,1.000000,0.298712,0.495688,62.962101,62.962101
827.200000,nan,0.000000,nan,nan,nan,nan
829.400000,0.000000,1.000000,0.283123,nan,24.236029,24.236029
830.000000,nan,0.000000,nan,nan,nan,nan
831.200000,0.000000,1.000000,0.306179,0.478816,93.578626,93.578626
832.200000,0.000000,1.000000,0.306874,nan,96.901914,96.901914
832.600000,nan,0.000000,nan,nan,nan,nan
